In [1]:
from ase import Atoms
import ase.units as units
from ase.io import read
from ase.build import bulk
from ase.calculators.espresso import Espresso

import numpy as np
import matplotlib.pyplot as plt

import os
import dotenv

dotenv.load_dotenv()
PWSCF_COMMAND = os.environ.get("PWSCF_COMMAND")
PPDIR = os.environ.get("PSEUDOPOTENTIALS")

outdir = "./results"
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [2]:
from mp_api.client import MPRester
from pymatgen.io.ase import AseAtomsAdaptor

MP_API_KEY = os.environ.get("MP_API_KEY")

def get_cif(material_id):
    m = MPRester(MP_API_KEY)
    structure = m.get_structure_by_material_id(material_id)
    return structure

KPbI3 = get_cif("mp-1211504")  # Replace "mp-149" with your material id
AAA = AseAtomsAdaptor()
lattice = AAA.get_atoms(KPbI3)


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# Setup DFT calculation parameters
pseudopotentials = {'K': 'K.pbesol-spn-kjpaw_psl.1.0.0.UPF',
                    'Pb': 'Pb.pbesol-dn-kjpaw_psl.0.2.2.UPF',
                    'I': 'I.pbesol-n-kjpaw_psl.0.2.UPF'}  # Example, change as needed

k_points = 8
calc_params = {
    'calculation': 'scf',
    'tprnfor': True,
    'tstress': True,
    'input_data': {
        'system': {
            'ecutwfc': 30 * units.eV,  # PW cutoff
            'ecutrho': 240 * units.eV,  # Charge cutoff
            'ibrav': 0
        },
        'electrons': {
            'diagonalization': 'david',
            'mixing_beta': 0.5,
            'conv_thr': 1e-07,
        }
    },
    'pseudopotentials': pseudopotentials,
    'kpts': (k_points, k_points, k_points),  # k-points grid, adjust as needed
    'parallel': 'all',
    'directory': outdir,  # Custom directory for calculation files
    'label': "KPbI",  # Prefix for the filenames
    'logfile': "KPbI.log",  # Logfile name
    'command': "mpirun -np 8 " + PWSCF_COMMAND + " -in KPbI.pwi > KPbI.pwo",
    'pseudo_dir': PPDIR,
}

In [6]:
# Attach the calculator to the structures
calculator = Espresso(**calc_params)

print (calc_params['command'])
lattice.set_calculator(calculator)
energy_bcc = lattice.get_potential_energy()

final_structure = read('results/KPbI.pwo') 

mpirun -np 8 /opt/local/bin/pw.x -in KPbI.pwi > KPbI.pwo


KeyboardInterrupt: 